In [ ]:
from pathlib import Path
import cv2
from PIL import Image
import pytesseract
from corpus_unpdf import get_img
ROI = "temp/roi.png"
PAGE_NUM = 0
PDF = Path().cwd() / "data" / "data2.pdf"
im = get_img(PDF, pagenum=PAGE_NUM)

In [ ]:
import numpy
opencvImage = cv2.cvtColor(numpy.array(im), cv2.COLOR_RGB2BGR)
base_image = opencvImage.copy()
im_h, im_w, im_d = opencvImage.shape

In [ ]:
from corpus_unpdf.images.pdf_lines import get_contours
im_h, im_w, _ = opencvImage.shape
for c in get_contours(opencvImage, (30, 10)):
    x, y, w, h = cv2.boundingRect(c)
    if w > 1200:
      print(f"{x}, {y}, {w}, {h}: {(y + h) / im_h}")
      cv2.rectangle(opencvImage, (x,y), (x+w, y+h), (36, 255, 12), 3)
cv2.imwrite("temp/sample_boxes.png", opencvImage)

In [ ]:
import pdfplumber
pdf = pdfplumber.open(PDF)
page = pdf.pages[0]
page.crop(
  bbox=(0, page.height * start_decision_bottom, page.width, page.height),
  relative=False,
  strict=True,
).to_image()

In [ ]:
if footnote_top:
  page.crop(
    bbox=(0, (page.height * footnote_top) + 5, page.width, page.height * 0.938),
    relative=False,
    strict=True,
  ).to_image()

In [ ]:
from PIL import Image
try: # roi found
    cv2.imwrite(ROI, roi)
    ocr_result = pytesseract.image_to_string(roi)
    pilimg = Image.open(ROI)
    pilimg.save(
        "temp/temp.pdf", "PDF" ,resolution=100.0, save_all=True
    )	
    print("Reveal ROI")    

except NameError: # no roi, just remove header
    cv2.imwrite(ROI, base_image)
    ocr_result = pytesseract.image_to_string(base_image[HEADER_TOP:im_h, 0:im_w])
    pilimg = Image.open(ROI)
    pilimg.save(
        "temp/temp.pdf", "PDF" ,resolution=300.0, save_all=True
    )	
    print("Reveal base")


In [ ]:
ocr_result

In [ ]:
import pdfplumber
pdf = pdfplumber.open("temp/temp.pdf")

In [ ]:
pdf.pages[0].extract_text()